In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
import time
import datetime 
%matplotlib inline  
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/jz2308/bus-Capstone')

# these two modules are homemade
import gtfs
import arrivals
import ttools
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

In [2]:
base = datetime.date(2015, 1, 1)
schedule_samples = [str(base + datetime.timedelta(days=x)) for x in range(0, 365)]
#schedule_samples = ['2015-01-04','2015-04-05','2015-06-27','2015-07-06','2015-09-05','2015-09-15','2015-10-12']
collected = pd.DataFrame()
for sdate in schedule_samples[0:1]:
    stops = gtfs.load_stops(sdate,'gtfs/')
    stops['schedule_sample_date'] = sdate
    collected = collected.append(stops[['stop_lat','stop_lon','stop_name','schedule_sample_date']].drop_duplicates())

In [5]:
collected.reset_index().head()

,stop_id,stop_lat,stop_lon,stop_name,schedule_sample_date
0,300000,40.578209,-73.939697,ORIENTAL BL/MACKENZIE ST,2015-01-01
1,300002,40.578068,-73.943031,ORIENTAL BL/JAFFRAY ST,2015-01-01
2,300003,40.577961,-73.944664,ORIENTAL BL/HASTINGS ST,2015-01-01
3,300004,40.577721,-73.946205,ORIENTAL BL/FALMOUTH ST,2015-01-01
4,300006,40.577354,-73.949554,ORIENTAL BL/DOVER ST,2015-01-01


In [3]:
# get all the schedule data. (subset can be created later)
trips = gtfs.load_trips('2015-12-03','gtfs/')
# stops = gtfs.load_stops('2015-12-03','gtfs/')
stop_times, tz_sched = gtfs.load_stop_times('2015-12-03','gtfs/')
print 'Finished loading GTFS data.'
joined = stop_times.reset_index(level=1)[['stop_id','stop_sequence']].join(trips[['route_id','direction_id']])
unique_stops = joined.reset_index().drop_duplicates(subset=['route_id','direction_id','stop_id','stop_sequence']).drop('trip_id',axis=1)
unique_stops = unique_stops.set_index(['route_id','stop_id'],verify_integrity=False)
dupe_inds = unique_stops.index[unique_stops.index.duplicated()]
unique_stops = unique_stops.loc[dupe_inds]

Finished loading GTFS data.


In [12]:
unique_stops.reset_index().tail()

,route_id,stop_id,stop_sequence,direction_id
63221,S44,200047,60,0
63222,S44,200047,56,0
63223,S44,805016,14,0
63224,S44,805016,61,0
63225,S44,805016,57,0


In [11]:
collected.reset_index().head()

,stop_id,stop_lat,stop_lon,stop_name,schedule_sample_date
0,300000,40.578209,-73.939697,ORIENTAL BL/MACKENZIE ST,2015-01-01
1,300002,40.578068,-73.943031,ORIENTAL BL/JAFFRAY ST,2015-01-01
2,300003,40.577961,-73.944664,ORIENTAL BL/HASTINGS ST,2015-01-01
3,300004,40.577721,-73.946205,ORIENTAL BL/FALMOUTH ST,2015-01-01
4,300006,40.577354,-73.949554,ORIENTAL BL/DOVER ST,2015-01-01


In [13]:
collected.reset_index().join(unique_stops.reset_index())

ValueError: columns overlap but no suffix specified: Index([u'stop_id'], dtype='object')

In [ ]:
collected.tail()

In [ ]:
reliability_metrics = pd.read_csv('performance.csv',names = ["Line", "stop_id", "schedule_sample_date", "ontime_ratio", "peak_hours wait ass","offpeak hours wait ass"])

In [ ]:
collected.reset_index().head()

In [ ]:
pd.DataFrame(reliability_metrics.groupby('stop_id').mean().reset_index()).head()

In [ ]:
stops_metrics = collected.reset_index().merge(pd.DataFrame(reliability_metrics.groupby('stop_id').mean().reset_index()))
stops_metrics = stops_metrics.groupby('stop_id').mean()[['stop_lat','stop_lon','ontime_ratio','offpeak hours wait ass']]

In [ ]:
stops_metrics.head()

In [ ]:
stops_metrics.to_csv('metrics-by-stop-2015.csv')

In [ ]:
line_metrics = collected.reset_index().merge((pd.DataFrame(reliability_metrics.groupby(['Line','stop_id']).mean().reset_index())))

In [ ]:
line_metrics.head()

In [ ]:
line_metrics.query("Line=='B41'").to_csv('metrics-by-line-stop-B41-2015.csv')

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(40,20))
plt.scatter(stops_metrics.stop_lon,stops_metrics.stop_lat,c=stops_metrics.ontime_ratio,s=55)
